In [102]:
import os
import pyproj
import csv
import pandas as pd

# 1. 파일 위치에 union할 파일명 가져와서 union 시키기

# Lambda 형 활용!! (중요)
match_list = list(filter(lambda x: "실거래조회"in x, os.listdir('./')))

# match_list

df_data = pd.concat([pd.read_csv(f, encoding= 'EUC-KR') for f in match_list])
df_data.reset_index(inplace=True, drop=True)

# df_data.head(5)

df_data.columns


Index(['번호', '실거래정보코드', '신고유형', '거래원인', '물건구분', '계약일자', '신고일자', '지번주소', '상세주소',
       '단지명', '동', '호', '토지면적(㎡)', '건물면적(㎡)', '거래금액(원)', '용도지역', '지목', '건물유형',
       '지분여부', '거래지분', '매수자유형', '매도자유형', '매수자거주지', 'X좌표', 'Y좌표', '지번코드',
       '공부면적(㎡)', '용도지구', '세부토지용도', '이용상황', '고저', '형상', '방위', '가로조건', '도로거리',
       '지가(원/㎡)', '1년전지가(원/㎡)', '2년전지가(원/㎡)', '3년전지가(원/㎡)', '4년전지가(원/㎡)'],
      dtype='object')

In [103]:
# 2. 정상적으로 union 됐는지 확인

df_data['계약일자'].str[:4].unique()

array(['2016', '2017', '2018', '2021', '2020', '2019'], dtype=object)

In [135]:
# 3. X좌표, Y좌표 변환 (X좌표 = lng 경도, Y좌표 = lat 위도))

# 한국 주요 좌표계를 확인하여 실거래가에 나와 있는 X,Y 좌표가 어떠한 형식인지 확인 (epsg5174 라는걸 확인)
# epsh5174라는걸 확인하는건 도움을 받았음..

# 해당 URL에서 epsg5174를 계산하는 식을 가져와 적용 (https://www.osgeo.kr/17)

# epsg:4326은 pyproj를 사용하고 epsg5174는 그냥 사용하는 이유는 약간 다른거 같음


# epsg5174 계산식 적용
epsg5174 = pyproj.Proj(f'''
    +proj=tmerc +lat_0=38 +lon_0=127.0028902777778 +k=1 +x_0=200000 +y_0=500000
    +ellps=bessel +units=m +no_defs 
    +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43
''')

epsg4326 = pyproj.Proj('epsg:4326')
transformer = pyproj.Transformer.from_proj(epsg5174, epsg4326)

list_lng = []
list_lat = []

# 경위도 변환 결과 테스트
(trans_lng, trans_lat) = transformer.transform(df_data['X좌표'][0], df_data['Y좌표'][0])
print(df_data['지번주소'][0], ': \t', trans_lng, '\t', trans_lat, '\t', df_data['X좌표'][0], '\t', df_data['Y좌표'][0])


for i in range(0, len(df_data)) :

    (trans_lng, trans_lat) = transformer.transform(df_data['X좌표'][i], df_data['Y좌표'][i])

    list_lng.append(trans_lng)
    list_lat.append(trans_lat)


df_data['lng'] = list_lng
df_data['lat'] = list_lat

df_data.loc[:, ['지번코드', 'X좌표', 'Y좌표', 'lng','lat']]

경기도 광명시 광명동 11-20 : 	 37.48742828350402 	 126.85998313886667 	 187547.0 	 442813.0


,지번코드,X좌표,Y좌표,lng,lat
0,4121010100100110020,187547.0,442813.0,37.487428,126.859983
1,4121010100100110105,187495.0,442764.0,37.486986,126.859396
2,4121010100100240095,187679.0,442071.0,37.480745,126.861488
3,4121010100100240113,187680.0,442035.0,37.480420,126.861500
4,4121010100100270089,187717.0,441998.0,37.480087,126.861919
...,...,...,...,...,...
6776,4121010800200210016,185219.0,441186.0,37.472735,126.833693
6777,4121010800200210017,185234.0,441143.0,37.472347,126.833864
6778,4121010800201160001,184816.0,440149.0,37.463385,126.829159
6779,4121010800201160001,184819.0,440154.0,37.463430,126.829193


In [137]:
df_data.to_csv('output.csv', index=False, encoding='euc-kr')